In [1]:
import pandas as pd
from src.preprocessing import Preprocessor

prep = Preprocessor()
prep.full_prep()
news_df = prep.df

In [5]:
print(news_df['source'].nunique())
print(news_df['source'].value_counts(True))

1359
source
Yahoo              0.198135
Reuters            0.148945
BBC                0.099642
New                0.080379
Washington         0.042290
                     ...   
just-food.com      0.000013
Skipressworld      0.000013
Destination        0.000013
SaddoBoxing.com    0.000013
Kinston            0.000013
Name: proportion, Length: 1359, dtype: float64


In [ ]:
news_df["source"].value_counts().describe()

count     1359.000000
mean        58.863870
std        624.941435
min          1.000000
25%          1.000000
50%          2.000000
75%          8.000000
max      15850.000000
Name: count, dtype: float64

In [17]:
for label in sorted(news_df['label'].unique()):
    sub_news_df = news_df[news_df['label'] == label]
    print('='*30)
    print(label)
    print(sub_news_df['source'].value_counts(True)[:10])
    print('='*30)

0
source
Yahoo         0.357829
BBC           0.186942
Reuters       0.134526
Topix.Net     0.056537
New           0.049444
Washington    0.035384
RedNova       0.022938
Boston        0.018180
CSMonitor     0.013041
Time          0.010407
Name: proportion, dtype: float64
1
source
Reuters          0.219514
New              0.129026
Yahoo            0.089449
Motley           0.085104
International    0.052139
Forbes           0.051006
Washington       0.047606
BBC              0.044205
Boston           0.043827
Ananova          0.028998
Name: proportion, dtype: float64
2
source
RedNova          0.175522
Yahoo            0.164501
CNET             0.118807
New              0.082967
Register         0.073739
Computerworld    0.041394
InfoWorld        0.037900
PCWorld          0.036645
Wired            0.031270
Reuters          0.030911
Name: proportion, dtype: float64
3
source
Yahoo         0.222913
Reuters       0.128496
New           0.123684
BBC           0.040493
Syfy.com      0.028165


In [19]:
top_k = news_df["source"].value_counts()[:50].index
(
 news_df[news_df["source"].isin(top_k)]
 .groupby("source")["label"]
 .mean()
 .sort_values()
)


source
Topix.Net          0.000000
CSMonitor          0.838028
Motley             1.061224
Ananova            1.078616
Yahoo              1.403407
International      1.529101
RedNova            1.674935
Forbes             1.714882
Wired              1.776718
InfoWorld          1.777978
\N                 1.846939
BBC                1.850458
Computerworld      1.993548
BCC                2.000000
PCWorld            2.000000
Register           2.021583
CNET               2.029949
Search             2.033898
Boston             2.281682
Channel            2.289100
New                2.330949
Daily              2.418750
Australian         2.550898
Reuters            2.564666
Bloomberg          2.611794
Washington         2.653858
Xinhua             2.689024
Scotsman           2.768559
Independent        2.852713
Times              2.865116
Voice              2.867159
Time               2.871758
CBC                2.950495
Telegraph.co.uk    2.968354
Syfy.com           3.000000
ABC          

Honestly, I'm not able to understand if there is a correlation between source and labels because there are too much sources; my knowledge in the topic tells me that there is a correlation, but due to the fact that OHE it's pretty useless and explode having ~=1350 different sources, I will OHE only the most common sources.

In [24]:
news_df["source"].value_counts(True)[:50].sum()

np.float64(0.8675933796689835)

The top 50 sources accounts for 86% of the articles; I think that's enough.  
I will create 50 columns and all the other sources will be encoded with all 0s

In [25]:
from sklearn.preprocessing import OneHotEncoder as OHE

In [41]:

ohe = OHE(
    max_categories=50,
    sparse_output=False
)

In [42]:
X = news_df['source'].values

In [43]:
ohe.fit_transform(X.reshape(-1,1))

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], shape=(79996, 50))

In [ ]:
ohe.get_feature_names_out()

array(['x0_ABC', 'x0_Ananova', 'x0_Australian', 'x0_BBC', 'x0_BCC',
       'x0_Bloomberg', 'x0_Boston', 'x0_CBC', 'x0_CBS', 'x0_CNET',
       'x0_CNN', 'x0_CSMonitor', 'x0_Canada.com', 'x0_Channel',
       'x0_Computerworld', 'x0_Daily', 'x0_ESPN', 'x0_Forbes',
       'x0_Guardian', 'x0_Houston', 'x0_Independent', 'x0_InfoWorld',
       'x0_International', 'x0_Motley', 'x0_New', 'x0_News', 'x0_Newsday',
       'x0_Newsweek', 'x0_PCWorld', 'x0_RedNova', 'x0_Rediff',
       'x0_Register', 'x0_Reuters', 'x0_San', 'x0_Scotsman', 'x0_Seattle',
       'x0_Sports', 'x0_Syfy.com', 'x0_Telegraph.co.uk', 'x0_Time',
       'x0_Times', 'x0_Topix.Net', 'x0_USA', 'x0_Voice', 'x0_Washington',
       'x0_Wired', 'x0_Xinhua', 'x0_Yahoo', 'x0_\\N',
       'x0_infrequent_sklearn'], dtype=object)

---

IMPORTANT: I notice now that there was a maskered NaN categorie as \N.  
I've now replaced that when loading the ds, so I will restart the notebook but change only cells after this.

In [8]:
news_df[news_df['source'].isna()]['timestamp'].value_counts()

timestamp
1970-01-01 00:00:00    100
2004-11-10 16:39:29      1
2006-09-06 17:07:43      1
2007-03-12 15:52:43      1
2006-08-30 18:06:07      1
                      ... 
2004-10-01 21:26:47      1
2007-12-20 21:41:27      1
2007-03-13 15:51:29      1
2007-02-26 11:08:52      1
2007-01-31 12:45:27      1
Name: count, Length: 195, dtype: int64

In [9]:
news_df.isna().sum()

source        294
title           0
article      1874
page_rank       0
timestamp       0
label           0
dayofweek       0
month           0
hour            0
year            0
dtype: int64

In [14]:
from sklearn.preprocessing import OneHotEncoder as OHE
ohe = OHE(
    max_categories=50,
    sparse_output=False
)
X = news_df['source'].values
X_ohe = ohe.fit_transform(X.reshape(-1,1))
X_ohe_df = pd.DataFrame(X_ohe, columns=ohe.get_feature_names_out())

In [18]:
X_ohe_df[X_ohe_df.sum(axis=1) == 0]

,x0_ABC,x0_Ananova,x0_Australian,x0_BBC,x0_BCC,x0_Bloomberg,x0_Boston,x0_CBC,x0_CBS,x0_CNET,...,x0_Times,x0_Topix.Net,x0_USA,x0_Voice,x0_Washington,x0_Wired,x0_Xinhua,x0_Yahoo,x0_nan,x0_infrequent_sklearn


This is not what I want... I want:

- ohe for top50 categories (THAT INCLUDE ALSO a NaN category; for this first attempt, I will use it as a "special" source)
- For infrequent categories => all 0s

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder as OHE

news_df.replace(np.nan, 'MISSING', inplace=True)
top_50 = news_df["source"].value_counts()[:50].index

X = news_df["source"].where(news_df["source"].isin(top_50), np.nan).to_frame()

ohe = OHE(
    categories=[top_50],
    handle_unknown="ignore",
    sparse_output=False
)

X_ohe = ohe.fit_transform(X)
X_ohe_df = pd.DataFrame(X_ohe,columns=ohe.get_feature_names_out(),index=news_df.index)

Maybe it's a workaround, but it works.

In [48]:
X_ohe_df.sum().sum() / news_df.shape[0]

np.float64(0.8675933796689834)